### Cifar10 convolutional neural network

Loading and preparing data

In [10]:
import tensorflow as tf
import numpy as np

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [8]:
print(f"Dataset training size X={X_train.shape}, Y={y_train.shape}, validation size X={X_test.size}, Y={y_test.shape}")

Dataset training size X=(50000, 32, 32, 3), Y=(50000, 1), validation size X=30720000, Y=(10000, 1)


In [9]:
X_train_scaled = X_train / 255
X_test_scaled =  X_test / 255

In [12]:
print(f"Unique output are: {np.unique(y_train)}, where every number represents different class")

Unique output are: [0 1 2 3 4 5 6 7 8 9], where every number represents different class


In [13]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=10, dtype=np.float32)
y_test_one_hot = to_categorical(y_test, num_classes=10, dtype=np.float32)

In [17]:
from keras.layers import Flatten, Layer, Conv2D, MaxPooling2D, Dense
from keras import Sequential

simple_conv_model = Sequential([
    Conv2D(32, (5,5), activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Conv2D(64, (5,5), strides=(1,1)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Flatten(),
    Dense(400, activation='relu'),
    Dense(120, activation='relu'),
    Dense(40, activation='relu'),
    Dense(10, activation='softmax')
])

simple_conv_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

2022-06-23 00:56:19.492116: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 00:56:19.495626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 00:56:19.495759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 00:56:19.496458: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [18]:
simple_conv_model.fit(X_train_scaled, y_train_one_hot, batch_size=256, validation_data=(X_test_scaled, y_test_one_hot), epochs=100)

Epoch 1/100


2022-06-23 00:58:03.683154: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-06-23 00:58:04.820407: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-23 00:58:04.820889: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-23 00:58:04.820908: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-06-23 00:58:04.821387: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-23 00:58:04.821438: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


196/196 [==============================] - 4s 8ms/step - loss: 1.8555 - accuracy: 0.3302 - val_loss: 1.6936 - val_accuracy: 0.4130
Epoch 2/100
196/196 [==============================] - 1s 6ms/step - loss: 1.4783 - accuracy: 0.4772 - val_loss: 1.5139 - val_accuracy: 0.4694
Epoch 3/100
196/196 [==============================] - 1s 6ms/step - loss: 1.2663 - accuracy: 0.5548 - val_loss: 1.3039 - val_accuracy: 0.5461
Epoch 4/100
196/196 [==============================] - 1s 6ms/step - loss: 1.1050 - accuracy: 0.6135 - val_loss: 1.1383 - val_accuracy: 0.5970
Epoch 5/100
196/196 [==============================] - 1s 6ms/step - loss: 0.9818 - accuracy: 0.6595 - val_loss: 1.1536 - val_accuracy: 0.6050
Epoch 6/100
196/196 [==============================] - 1s 6ms/step - loss: 0.8743 - accuracy: 0.6946 - val_loss: 1.0554 - val_accuracy: 0.6283
Epoch 7/100
196/196 [==============================] - 1s 6ms/step - loss: 0.7822 - accuracy: 0.7309 - val_loss: 1.1305 - val_accuracy: 0.6165
Epoch 8/100

First model evaluation is showing ENORMOUS overfitting after 100 epochs loss on train set is 0.03 wheras on validation set it is over 5. To prevent this we are going to apply dropout regularization and create EarlyStopping instance to stop learning after certain point

In [19]:
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

simple_conv_model_v2 = Sequential([
    Conv2D(32, (5,5)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Conv2D(64, (5,5), strides=(1,1)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Flatten(),
    Dense(400, activation='relu'),
    Dropout(0.5),
    Dense(120, activation='relu'),
    Dense(40, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

simple_conv_model_v2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1)